# Pumas NCA Tutorial - Analyzing infusion data
### Beatriz Guglieri Lopez, Shamir Kalaria, Vijay Ivaturi
### July 19, 2019

In [ ]:
using Pumas, PumasTutorials, CSV, Plots

# Introduction

In this tutorial, we will cover the fundamentals of performing an NCA analysis
with Pumas using an example dataset in which an intravenous infusion was administered.

# The dataset

* Single 2000 mg 2-hour IV infusion dose to 24 subjects
* Blood samples for pharmacokinetic analysis were collected every 30 minutes

In [ ]:
data = PumasTutorials.tutorial_data("data/nca","SD_IV_infusion")
data = CSV.read(data,missingstring="NA")
first(data,10)

Please, note that the `DOSE` and the `Infusion_Time` column must only contain a value at the time the dose was administered.

# Defining the units

In [ ]:
timeu = u"hr"
concu = u"mg/L"
amtu  = u"mg"

# Defining the population object

In the case of the infusion, the `read_nca` function should

* carry the name of the column that contains the duration of the infusion (`duration=`).

In [ ]:
pop = read_nca(data, id=:ID, time=:time, conc=:DV, amt=:DOSE, ii=24timeu,
    route=:Formulation, duration=:Infusion_Time,timeu=timeu,
    concu=concu, amtu=amtu,lloq=0.4concu)

Note that in the above syntanx:

* `route=` is mapped to the `Formulation` column that should specify `iv`
* LLOQ was set to 0.4 by `llq=0.4concu`

A basic plot function exists for single dose data without grouping or multiple analytes.
More functionality will be added soon. In this example of single dose data, here is the plot output

In [ ]:
plot(pop)

# NCA functions

### `NCA.auc`

In [ ]:
NCA.auc(pop,auctype=:last,method=:linear)

To change the methods to log-linear trapezoidal (`method=:linuplogdown`) or to linear-log (`method=:linlog`) one can use

In [ ]:
NCA.auc(pop,auctype=:inf,method=:linuplogdown)

To compute the AUC over an interval, one could do

In [ ]:
NCA.auc(pop, interval=(0,12).*timeu)

where we need to apply the time unit (`timeu`) to the `interval` for units compatibility. Multiple intervals can also be specified:

In [ ]:
NCA.auc(pop, interval=[(0,12).*timeu,(0,6).*timeu])

### `NCA.lambdaz`

The function to calculate the terminal rate constant (λz) is:

In [ ]:
NCA.lambdaz(pop)

This function has options that allow
*  to specify the maximum number of points to be used for lambdaz - `threshold=3`
* calculation to be peformed over specified indices - `idxs=[18,19,20]` where index 18,19,20 of the subject will be used for lambdaz
* speification of exact time points to use for lambdaz - `slopetimes=[18.5,19,19.5].*timeu`

In [ ]:
NCA.lambdaz(pop, threshold=3)
NCA.lambdaz(pop, idxs=[18,19,20])
NCA.lambdaz(pop, slopetimes=[18.5,19,19.5].*timeu)

### `NCA.cmax`

To calculate the maximum concentration for the first subject we would use:

In [ ]:
cmax = NCA.cmax(pop[1])

### ` NCA.normalizedose`

If we want dose-normalized Cmax for that same subject:

In [ ]:
NCA.normalizedose(cmax,pop[1])

This can be used on any parameter that can be dose normalized.

Other functions to calculate single PK parameters are the following:

In [ ]:
NCA.lambdazr2(pop)
NCA.lambdazadjr2(pop)
NCA.lambdazintercept(pop)
NCA.lambdaztimefirst(pop)
NCA.lambdaznpoints(pop)

NCA.tmax(pop)
NCA.cmin(pop)
NCA.tmin(pop)

NCA.tlast(pop)
NCA.clast(pop)

NCA.aumc(pop)
NCA.aumclast(pop)

NCA.thalf(pop)

NCA.cl(pop)

NCA.vss(pop)
NCA.vz(pop)

# NCA report

If we want a complete report of the NCA analysis we can just use the function `NCAreport` to obtain a data frame that contains
all the above mentioned pharmacokinetic parameters.

In [ ]:
report = NCAReport(pop)
report = NCA.to_dataframe(report)

By default, the AUC and AUMC reported are observed. If predicted PK parameters are needed instead, the following code should be used:

In [ ]:
report = NCAReport(pop,pred=true)
report = NCA.to_dataframe(report)

Finally, we can save this data frame as a csv file if desired.

In [ ]:
CSV.write("./tutorials/nca/report_SD_IV_infusion.csv", report)

In [ ]:
using PumasTutorials
PumasTutorials.tutorial_footer(WEAVE_ARGS[:folder],WEAVE_ARGS[:file])